In [1]:
import pandas as pd
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import load_workbook
import time
import datetime
pd.options.mode.chained_assignment = None  # default='warn'


In [2]:
def convert_to_int_transfer(cell):
    """
    Функция для конвертации в int
    """
    try:
        return int(cell)
    except ValueError:
        return 0

In [3]:
def create_doc_convert_date(cell):
    """
    Функция для конвертации даты при создании документов
    :param cell:
    :return:
    """
    try:
        string_date = datetime.datetime.strftime(cell, '%d.%m.%Y')
        return string_date
    except ValueError:
        return ''

In [29]:
group_rb_region_transfer_3_to_reestr = 1

In [46]:
# file_transfer_to_upp = 'data/Перенос из таблицы 3 в реестр УПП/Таблица 3 с дополнительными колонками..xlsx'
# file_transfer_reestr = 'data/Перенос из таблицы 3 в реестр УПП/2022-10-27_64_Реестр УПП с дополнительными колонками.xlsx'

file_transfer_to_upp = 'data/Перенос из таблицы 3 в реестр УПП/Якутия октябрь_Таблица 3с доп. колонками.xlsx'
file_transfer_reestr = 'data/Перенос из таблицы 3 в реестр УПП/Якутия 2022-11-27_72_Реестр УПП по новой форме.xlsx'

path_to_end_folder = 'data/'
header_reestr = 'data/Перенос из таблицы 3 в реестр УПП/Заголовок Реестр УПП от 25.10.xlsx'

In [47]:
# Загружаем датафреймы
df_upp = pd.read_excel(file_transfer_reestr,skiprows=8)
df_table_3 = pd.read_excel(file_transfer_to_upp,skiprows=6,usecols=[4,5,6,7,8,9,10,12,13,14,15,16,17,18,19,21,22,23,24,25,26,27,28,
                                                           29,32])


# Приводим названия колонок к строковому виду, чтобы избежать возможных проблем с названиями колонок
df_upp.columns = list(map(str,list(df_upp.columns)))
df_table_3.columns = list(map(str,list(df_table_3.columns)))

# Очищаем на всякий случай от пробельных символов
df_upp.columns = list(map(lambda x:x.strip(),list(df_upp.columns)))
df_table_3.columns = list(map(lambda x:x.strip(),list(df_table_3.columns)))

# Фильтруем датафрейм отбирая только те записи в которых есть 1 в графе 30
transfer_df=df_table_3[df_table_3['30'] == 1]


# перемещаем площадь выдела
transfer_df.insert(7,'12',transfer_df['33'])

# удаляем лишний столбец с площадью выдела и признаков внесения в реестр
transfer_df.drop(columns=['30','33'],inplace=True)

transfer_df['17'] =transfer_df['17'].astype(str) # Приводим колонку к строковому формату

# Заменяем категории таблицы 3 на категории Реестра УПП
transfer_df['17'] =transfer_df['17'].replace(regex={'120':'2','100':'1',r'130|131|132|133|134|135|136':'3',
                                                    r'141|142|143|144|145|146|147|148|149|150|151|152':'4','nan':'0'})

transfer_df.columns = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24']

# Создаем датафрейм для проверки наличия записи в реестре
checked_df = df_upp[['1','2','3','4','5']]
checked_df = checked_df.astype(str) # делаем данные строковыми
# Создаем в каждом датафрейме колонку с айди путем склеивания всех нужных колонок в одну строку
checked_df['ID'] = checked_df.loc[:,['1','2','3','4','5'] ].sum(axis=1)
# Удаляем пробелы
checked_df['ID'] = checked_df['ID'].apply(lambda x:x.replace(' ',''))


# делаем строковыми первые 5 колонок
transfer_df[['1','2','3','4','5',]] = transfer_df[['1','2','3','4','5',]].astype(str)
transfer_df['ID'] = transfer_df.loc[:,['1','2','3','4','5'] ].sum(axis=1)
transfer_df['ID'] = transfer_df['ID'].apply(lambda x:x.replace(' ',''))

# Мержим по полю айди
merged_df = pd.merge(checked_df,transfer_df,how='outer',left_on='ID',right_on='ID',indicator=True)
merged_df.to_excel('Тест.xlsx',index=False)

# Отбираем только те значения которые есть в правом датафрейме
added_df = merged_df[merged_df['_merge']=='right_only']

added_df.drop(columns=['1_x','2_x','3_x','4_x','5_x','ID','_merge'],inplace=True) # удаляем лишние колонки

added_df.rename(columns={'1_y':'1','2_y':'2','3_y':'3','4_y':'4','5_y':'5'},inplace=True) # переименовываем колонки для корректного добавления

itog_df = pd.concat([df_upp,added_df],ignore_index=True)

itog_df.sort_values(['1','2','3','4','5',
                     ],inplace=True)

# Приводим даты к нормальному виду ДД.ММ.ГГГГ
itog_df['17'] = pd.to_datetime(itog_df['17'],errors='coerce',dayfirst=True)
itog_df['22'] = pd.to_datetime(itog_df['22'],errors='coerce',dayfirst=True)
itog_df['17'] = itog_df['17'].apply(create_doc_convert_date)
itog_df['22'] = itog_df['22'].apply(create_doc_convert_date)


# конвертируем в инт номера квартала и выдела
itog_df['4'] = itog_df['4'].apply(convert_to_int_transfer)
itog_df['5'] = itog_df['5'].apply(convert_to_int_transfer)

# Получаем текущую дату
current_time = time.strftime('%H_%M_%S %d.%m.%Y')
# Сохраняем отчет
# Для того чтобы увеличить ширину колонок для удобства чтения используем openpyxl
wb = load_workbook(header_reestr) # Создаем объект
# Записываем результаты
for row in dataframe_to_rows(itog_df,index=False,header=False):
    wb['Реестр УПП'].append(row)

# Переименовываем после соединения колонки в таблице с добавленными участками
added_df.rename(columns={'1':'Лесничество','2':'Участковое лесничество','3':'Урочище','4':'Номер лесного квартала','5':'Номер лесотаксационного выдела',},inplace=True)

# Сохраняем файл с добавляемыми данными, чтобы пользователи знали что именно добавилось
added_df.to_excel(f'{path_to_end_folder}/Участки из таблицы 3 добавленные в реестр УПП от {current_time}.xlsx',index=False)

wb.save(f'{path_to_end_folder}/Реестр УПП с добавлением данных из таблицы 3 {current_time}.xlsx')

In [17]:
# Создаем список колонок которые нужно загрузить
use_cols = list(range(25))


In [24]:
# Загружаем датафреймы
df_upp = pd.read_excel(file_transfer_reestr,skiprows=8,usecols=use_cols)
df_table_3 = pd.read_excel(file_transfer_to_upp,skiprows=6,usecols=[4,5,6,7,8,9,10,12,13,14,15,16,17,18,19,21,22,23,24,25,26,27,28,
                                                           29,32])

In [25]:
# Приводим названия колонок к строковому виду, чтобы избежать возможных проблем с названиями колонок
df_upp.columns = list(map(str,list(df_upp.columns)))
df_table_3.columns = list(map(str,list(df_table_3.columns)))

# Очищаем на всякий случай от пробельных символов
df_upp.columns = list(map(lambda x:x.strip(),list(df_upp.columns)))
df_table_3.columns = list(map(lambda x:x.strip(),list(df_table_3.columns)))

In [26]:
df_table_3

,5,6,7,8,9,10,11,13,14,15,...,22,23,24,25,26,27,28,29,30,33
0,Намское,Хомустахское,NaN,323.0,8.0,37.0,NaN,NaN,NaN,1993.0,...,2022-03-21 00:00:00,1.0,Б,более 40%,более 40%,2022-05-16 00:00:00,1.0,0.0,1.0,1.0
1,Намское,Хомустахское,NaN,323.0,8.0,37.0,NaN,NaN,NaN,1993.0,...,2022-03-21 00:00:00,1.0,Б,более 40%,более 40%,2022-05-16 00:00:00,1.0,0.0,1.0,0.3
2,Намское,Хомустахское,NaN,323.0,8.0,37.0,NaN,NaN,NaN,1993.0,...,2022-03-21 00:00:00,1.0,Б,более 40%,более 40%,2022-05-16 00:00:00,1.0,0.0,1.0,5.3
3,Намское,Хомустахское,NaN,323.0,8.0,37.0,NaN,NaN,NaN,1993.0,...,2022-03-21 00:00:00,1.0,Б,более 40%,более 40%,2022-05-16 00:00:00,1.0,0.0,1.0,0.7
4,Намское,Хомустахское,NaN,323.0,8.0,37.0,NaN,NaN,NaN,1993.0,...,2022-03-21 00:00:00,1.0,Б,более 40%,более 40%,2022-05-16 00:00:00,1.0,0.0,1.0,6.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55725,Томпонское,Томпонское,NaN,140.0,8.0,17954.0,NaN,0.0,0.0,1949.0,...,01.07.2022-31.07.2022,1.0,Л,0.4,более 40%,NaN,1.0,0.0,1.0,1.9
55726,Томпонское,Томпонское,NaN,112.0,3.0,22542.0,NaN,0.0,0.0,1949.0,...,01.07.2022-31.07.2022,1.0,Л,0.4,более 40%,NaN,1.0,0.0,1.0,2.3
55727,Томпонское,Томпонское,NaN,112.0,3.0,22542.0,NaN,0.0,0.0,1949.0,...,01.07.2022-31.07.2022,1.0,Л,0.4,более 40%,NaN,1.0,0.0,1.0,22.3
55728,Томпонское,Томпонское,NaN,11.0,4.0,4896.0,NaN,0.0,0.0,1949.0,...,01.07.2022-31.07.2022,1.0,Л,0.4,более 40%,NaN,1.0,0.0,1.0,2.5


In [27]:


# Фильтруем датафрейм отбирая только те записи в которых есть 1 в графе 30
transfer_df=df_table_3[df_table_3['30'] == 1]


# перемещаем площадь выдела
transfer_df.insert(7,'12',transfer_df['33'])

# удаляем лишний столбец с площадью выдела и признаков внесения в реестр
transfer_df.drop(columns=['30','33'],inplace=True)

transfer_df['17'] =transfer_df['17'].astype(str) # Приводим колонку к строковому формату

# Заменяем категории таблицы 3 на категории Реестра УПП
transfer_df['17'] =transfer_df['17'].replace(regex={'120':'2','100':'1',r'130|131|132|133|134|135|136':'3',
                                                    r'141|142|143|144|145|146|147|148|149|150|151|152':'4','nan':'0'})

transfer_df.columns = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24']

In [45]:
# Дальше способы проверки наличия записи различаются для Бурятии и Якутии
# Бурятия
if group_rb_region_transfer_3_to_reestr == 0:
    # Создаем датафрейм для проверки наличия записи в реестре
    checked_df = df_upp[['1','2','3','4','5']]
    checked_df = checked_df.astype(str) # делаем данные строковыми
    # Создаем в каждом датафрейме колонку с айди путем склеивания всех нужных колонок в одну строку
    checked_df['ID'] = checked_df.loc[:,['1','2','3','4','5'] ].sum(axis=1)
    # Удаляем пробелы 
    checked_df['ID'] = checked_df['ID'].apply(lambda x:x.replace(' ',''))



    # делаем строковыми первые 5 колонок в таблице 3
    transfer_df[['1','2','3','4','5',]] = transfer_df[['1','2','3','4','5',]].astype(str)
    transfer_df['ID'] = transfer_df.loc[:,['1','2','3','4','5'] ].sum(axis=1)
    transfer_df['ID'] = transfer_df['ID'].apply(lambda x:x.replace(' ',''))

    # Мержим по полю айди
    merged_df = pd.merge(checked_df,transfer_df,how='outer',left_on='ID',right_on='ID',indicator=True)

    # Отбираем только те значения которые есть в правом датафрейме
    added_df = merged_df[merged_df['_merge']=='right_only']

    added_df.drop(columns=['1_x','2_x','3_x','4_x','5_x','ID','_merge'],inplace=True) # удаляем лишние колонки

    added_df.rename(columns={'1_y':'1','2_y':'2','3_y':'3','4_y':'4','5_y':'5'},inplace=True) # переименовываем колонки для корректного добавления

    itog_df = pd.concat([df_upp,added_df],ignore_index=True)

    itog_df.sort_values(['1','2','3','4','5',
                         ],inplace=True)

    # Приводим даты к нормальному виду ДД.ММ.ГГГГ
    itog_df['17'] = pd.to_datetime(itog_df['17'],errors='coerce',dayfirst=True)
    itog_df['22'] = pd.to_datetime(itog_df['22'],errors='coerce',dayfirst=True)
    itog_df['17'] = itog_df['17'].apply(create_doc_convert_date)
    itog_df['22'] = itog_df['22'].apply(create_doc_convert_date)


    # конвертируем в инт номера квартала и выдела
    itog_df['4'] = itog_df['4'].apply(convert_to_int_transfer)
    itog_df['5'] = itog_df['5'].apply(convert_to_int_transfer)

    # Получаем текущую дату
    current_time = time.strftime('%H_%M_%S %d.%m.%Y')
    # Сохраняем отчет
    # Для того чтобы увеличить ширину колонок для удобства чтения используем openpyxl
    wb = load_workbook(header_reestr) # Создаем объект
    # Записываем результаты
    for row in dataframe_to_rows(itog_df,index=False,header=False):
        wb['Реестр УПП'].append(row)

    # Переименовываем после соединения колонки в таблице с добавленными участками
    added_df.rename(columns={'1':'Лесничество','2':'Участковое лесничество','3':'Урочище','4':'Номер лесного квартала','5':'Номер лесотаксационного выдела',},inplace=True)

    # Сохраняем файл с добавляемыми данными, чтобы пользователи знали что именно добавилось
    added_df.to_excel(f'{path_to_end_folder}/Участки из таблицы 3 добавленные в реестр УПП от {current_time}.xlsx',index=False)

    wb.save(f'{path_to_end_folder}/Реестр УПП с добавлением данных из таблицы 3 {current_time}.xlsx')
# Якутия
elif group_rb_region_transfer_3_to_reestr == 1:
     # Создаем датафрейм для проверки наличия записи в реестре
    checked_df = df_upp[['1','2','3','4','5','11']]
    checked_df = checked_df.astype(str) # делаем данные строковыми
    # Создаем в каждом датафрейме колонку с айди путем склеивания всех нужных колонок в одну строку
    checked_df['ID'] = checked_df.loc[:,['1','2','3','4','5','11'] ].sum(axis=1)
     # Удаляем пробелы и нан
    checked_df['ID'] = checked_df['ID'].apply(lambda x:x.replace(' ',''))

        # делаем строковыми 6 колонок в таблице 3
    transfer_df[['1','2','3','4','5','11']] = transfer_df[['1','2','3','4','5','11']].astype(str)
    transfer_df['ID'] = transfer_df.loc[:,['1','2','3','4','5','11'] ].sum(axis=1)
    
    transfer_df['ID'] = transfer_df['ID'].apply(lambda x:x.replace(' ',''))
    
     # Мержим по полю айди
    merged_df = pd.merge(checked_df,transfer_df,how='outer',left_on='ID',right_on='ID',indicator=True)

    # Отбираем только те значения которые есть в правом датафрейме
    added_df = merged_df[merged_df['_merge']=='right_only']

    added_df.drop(columns=['1_x','2_x','3_x','4_x','5_x','11_x','ID','_merge'],inplace=True) # удаляем лишние колонки

    added_df.rename(columns={'1_y':'1','2_y':'2','3_y':'3','4_y':'4','5_y':'5','11_y':'11'},inplace=True) # переименовываем колонки для корректного добавления
    
    itog_df = pd.concat([df_upp,added_df],ignore_index=True)
    
    # Делаем колонку 11 с годом лесоустройства int
 

    itog_df.sort_values(['1','2','3','4','5','11'
                         ],inplace=True)
      # Приводим даты к нормальному виду ДД.ММ.ГГГГ
    itog_df['17'] = pd.to_datetime(itog_df['17'],errors='coerce',dayfirst=True)
    itog_df['22'] = pd.to_datetime(itog_df['22'],errors='coerce',dayfirst=True)
    itog_df['17'] = itog_df['17'].apply(create_doc_convert_date)
    itog_df['22'] = itog_df['22'].apply(create_doc_convert_date)


    # конвертируем в инт номера квартала и выдела
    itog_df['4'] = itog_df['4'].apply(convert_to_int_transfer)
    itog_df['5'] = itog_df['5'].apply(convert_to_int_transfer)

    # Получаем текущую дату
    current_time = time.strftime('%H_%M_%S %d.%m.%Y')
    # Сохраняем отчет
    # Для того чтобы увеличить ширину колонок для удобства чтения используем openpyxl
    wb = load_workbook(header_reestr) # Создаем объект
    # Записываем результаты
    for row in dataframe_to_rows(itog_df,index=False,header=False):
        wb['Реестр УПП'].append(row)

    # Переименовываем после соединения колонки в таблице с добавленными участками
    added_df.rename(columns={'1':'Лесничество','2':'Участковое лесничество','3':'Урочище','4':'Номер лесного квартала','5':'Номер лесотаксационного выдела','11':'Год лесоустройства'},inplace=True)

    # Сохраняем файл с добавляемыми данными, чтобы пользователи знали что именно добавилось
    added_df.to_excel(f'{path_to_end_folder}/Участки из таблицы 3 добавленные в реестр УПП от {current_time}.xlsx',index=False)

    wb.save(f'{path_to_end_folder}/Реестр УПП с добавлением данных из таблицы 3 {current_time}.xlsx')


0         1983
1         1983
2         1983
3         1983
4         1983
          ... 
283386       0
283387       0
283388       0
283389       0
283390     NaN
Name: 11, Length: 369094, dtype: object